In [ ]:
import json
import pandas as pd

In [ ]:
mainfile = "result_2023_6_8.json"

In [ ]:
maindata = []
with open(mainfile, 'r') as f:
    for entry in f:
        oneEntry = json.loads(entry)
        maindata.append(oneEntry)


Removing any redundancies

In [ ]:
filtereddata = []
for d in maindata:
    if d.get('report') != None:
        if d.get('report').get('user_id') != None:
            filtereddata.append(d)

In [ ]:
type(maindata)

Removing all non-GET request entries

In [ ]:
getrequests = []
for d in filtereddata:
    if d.get('report').get('request_method') == 'GET':      
        getrequests.append(d)

In [ ]:
print(len(maindata))
print(len(filtereddata))
print(len(getrequests))

In [ ]:
import copy

users = {}

emptydata = {
    "lastname":0,
    "firstname":0,
    "preferredname":0,
    "id":0,
    "email":0,
    "phone":0,
}


URLs = {}


user_data = {
    "URL":copy.deepcopy(emptydata),
    "body":copy.deepcopy(emptydata),
    "URLs":copy.deepcopy(URLs)
}


In [ ]:
for d in getrequests:
    user_id = d.get('report').get('user_id')
    if user_id not in users:
        users[user_id] = copy.deepcopy(user_data)
        print(user_id)

In [ ]:
for d in getrequests:
    user_id = d.get('report').get('user_id')
    url_leak_type = d.get('report').get('url_leak_type')
    body_leak_type = d.get('report').get('body_leak_type')
    domain = d.get('report').get('initiator_domain')
    
    
    if url_leak_type != None:
        for e in url_leak_type:
            users[user_id]["URL"][e] = users[user_id]["URL"][e] + 1
    
    if body_leak_type != None:
        for e in body_leak_type:
            users[user_id]["body"][e] = users[user_id]["body"][e] + 1
    
    if domain != None:
        if domain in users[user_id]["URLs"]:
            users[user_id]["URLs"][domain] += 1
        else:
            users[user_id]["URLs"][domain] = 1          

Just turning the above data into a nice table using tabulate

In [ ]:
from tabulate import tabulate

data = []
for u in users:
    entry = []
    entry.append(u)
    for e in users.get(u).get("URL"):
        entry.append(users.get(u).get("URL").get(e))
    
    data.append(entry)

headers = ['USER ID', 'Last Name', 'First Name','Pref. Name', 'ID', 'Email', "Phone #"]
table = tabulate(data, headers=headers, tablefmt='grid')
print(table)

Evidently, users "dknglbh1h3" and "zu7pjmkeoi" are outliers. Interestingly, this data doesn't quite line up with some older calculations for the number of leaks of each type, though those calculations probably didn't use the same filtering methodology I did. Everything's on the same order of magnitude, so it doesn't look like we have any problems. I've excluded those two users from the data set for the later analyses.

I'm also only showing data from the url_leak_type field, and not the body_leak_type field, even though I collected data from both. As it turns out, no personal information was found in the body_leak_type field. Possibly just encrypted, not absent, though. See below for the dictionaries.

(For what it's worth, rerunning this notebook from the top shuffles the order the USER IDs are presented in. Not sure why, but something to note down, I suppose.)

In [ ]:
for u in users:
    print(u + "\t" + str(users.get(u).get("body")) )
    
for u in users:
    print(u + "\t" + str(users.get(u).get("URLs")) )

In [ ]:
normalizeddata = []

for d in getrequests: 
    if d.get('report').get('user_id') != "dknglbh1h3" and d.get('report').get('user_id') != "zu7pjmkeoi":
        normalizeddata.append(d)

print(len(normalizeddata))            

In [ ]:
leaksOnlyNormalized = []

for d in normalizeddata: 
    if d.get('report').get('initiator_domain') != None:
        leaksOnlyNormalized.append(d)
        
print(len(leaksOnlyNormalized))            

The number of entries was reduced by 114,411 via filtering out of the two outlying users. There are 114,021 entries in which dknglbh1h3 or zu7pjmkeoi had a last name leak. Numbers seemingly track-- those two users had their last name "leaked" with nearly every search.

Filtering out entries without an initiator domain, we find that there are only 592 entries (down from the initial >100,000 leaked data entries detected-- so evidently those users were drastically skewing the data.

In [ ]:
import copy

sites = {}

leakCount = {
    "lastname":0,
    "firstname":0,
    "preferredname":0,
    "id":0,
    "email":0,
    "phone":0,
}

In [ ]:
initiatorDomains = []

for d in leaksOnlyNormalized:
    domain = d.get('report').get('initiator_domain')
    if domain not in sites:
        sites[domain] = copy.deepcopy(leakCount)
        print(domain)
        initiatorDomains.append(domain)


The list of initiator domains is worryingly short? Don't know if this pilot dataset is just that small or if something else is going on. Maybe an oversight on my part, but might just also mean we need a bigger dataset to make any more meaningful conclusions.

In [ ]:
for d in leaksOnlyNormalized:
    domain = d.get('report').get('initiator_domain')
    url_leak_type = d.get('report').get('url_leak_type')
    
    if url_leak_type != None:
        for e in url_leak_type:
            sites[domain][e] = sites[domain][e] + 1

In [ ]:
data = []
for d in sites:
    entry = []
    entry.append(d)
    for e in sites.get(d):
        entry.append(sites.get(d).get(e))
    
    data.append(entry)

headers = ['DOMAIN', 'Last Name', 'First Name','Pref. Name', 'ID', 'Email', "Phone #"]
table = tabulate(data, headers=headers, tablefmt='grid')
print(table)

This has already been said, but with internal trackers, I doubt this is the full extent of what data collection is occurring

In [ ]:
from urllib.parse import urlparse, urlunparse

def shorten(url):
    # Parse the URL into its components
    parsed_url = urlparse(url)

    # Reconstruct the URL with only the scheme, netloc, and fragment
    new_url = urlunparse((parsed_url.scheme, parsed_url.netloc, '', '', '', parsed_url.fragment))

    return new_url

In [ ]:
domainsAndLeaks = {}


for mainDomain in initiatorDomains:
    
    leakURLs = {}
    
    for d in leaksOnlyNormalized:
        domain = d.get('report').get('initiator_domain')
        if domain == mainDomain:
            leakURL = d.get('report').get('leak_url')
            leakURL = shorten(leakURL)
        
            if leakURL not in leakURLs:
                leakURLs[leakURL] = 1
            else:
                leakURLs[leakURL] += 1
                
    domainsAndLeaks[mainDomain] = leakURLs
    
print(domainsAndLeaks["https://www.google.com/"])
        

Some weird quirks here: for example, some of the URLs seem to be broken & including "LEAKED_FIRSTNAME" in places where it clearly shouldn't be. There's also an entire URL that seems to be wiped clean (I suspect that was marked Null in the original dataset, though how there's a leak without a Leak URL probably also warrants investigation). It might be due to the package I'm using to shorten the URLs, but I suspect it's more likely a relic of the scanner that needs to be fixed: maybe it should only be looking for leaks after the top level domain? Warrants discussion-- so for now I'll leave the error intact instead of manually paving over it until the cause is found

In [ ]:
for d in initiatorDomains:
    print(d)

In [ ]:
import os

value = input("Please request an initiator domain from the list above, or type 'exit' to close: ")

while True:
    
    
    if value == "exit":
        break
    data = domainsAndLeaks[value]
    table_data = [[key, value] for key, value in data.items()]

    table = tabulate(table_data, headers=["Attribute", "Value"], tablefmt="grid")
    print(table)
    print("\n\n")
    value = input("Please request an initiator domain, or type 'exit' to close: ")
    

This is a clunky little piece of code, and in future versions it might be better to just print everything, but I find this is a good way to make it easy to compare two specific URLs and keep the view short and sweet by focusing on what we determine relevant.